## Как не потерять градиент: функции активации, инициализация, нормализация

План на сегодня: разбираем возможные проблемы при обучении
1. Инициализация
   1. Диагностика проблем:
      - начальное распределение выхода
      - значения промежуточных активаций и нелинейности с насыщением
   2. Настраиваем инициализацию весов
2. Нормализация по батчам

Впереди нас ждут разные сложные архитектуры, но перед этим задержимся подольше на MLP, чтобы получить большьше интуитивного понимания об активациях и градиентах в нейронных сетях в процессе обучения



In [ ]:
import torch
from torch import nn, Tensor
from torch.nn import functional as F
import matplotlib.pyplot as plt

### 0. Подготовим данные, модель и функции для обучения

#### 0.1. Загружаем MNIST и создаём загрузчики данных:

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(
    'data', 
    train=True, 
    download=True,    
    transform=transforms.ToTensor(),
)
test_dataset = datasets.MNIST(
    'data', 
    train=False, 
    download=True,
    transform=transforms.ToTensor(),
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### 0.2. Задаём архитектуру модели

Такая же, как в прошлый раз, но параметры задаём явно и инициализируем значениями из $\mathcal{N}(0, 1)$

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
        super().__init__()
        self.w1 = nn.Parameter(torch.randn((input_dim, hidden_dim)), requires_grad=True)
        self.b1 = nn.Parameter(torch.randn(hidden_dim), requires_grad=True)
        self.w2 = nn.Parameter(torch.randn((hidden_dim, output_dim)), requires_grad=True)
        self.b2 = nn.Parameter(torch.randn(output_dim), requires_grad=True)

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        h = x.flatten(1) @ self.w1 + self.b1
        h_act = F.tanh(h)
        logits = h_act @ self.w2 + self.b2
        # помимо логитов, вернём ещё промежуточные активации - они нам понадобятся
        return logits, h_act, h


#### 0.3. Определим функции обучения

Всё как обычно, но
1. градиенты обновляем вручную
2. эпоху ограничиваем сотней батчей - просто для скорости

In [ ]:
def training_step(batch: tuple[torch.Tensor, torch.Tensor], model: MLP, lr: float = 0.01) -> torch.Tensor:
    # прогоняем батч через модель
    x, y = batch
    logits, *_ = model(x)
    # оцениваем значение ошибки
    loss = F.cross_entropy(logits, y)
    # обновляем параметры
    loss.backward()
    for param in model.parameters():
        if param.grad is not None:
            param.data -= lr * param.grad
        param.grad = None
    # возвращаем значение функции ошибки для логирования
    return loss

def train_epoch(dataloader: DataLoader, model: MLP, lr: float = 0.01, max_batches: int = 100) -> Tensor:
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        loss = training_step(batch, model, lr)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()

def test_epoch(dataloader: DataLoader, model: MLP, max_batches: int = 100) -> Tensor:
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        x, y = batch
        with torch.no_grad():
            logits, *_ = model(x)
        # оцениваем значение ошибки
        loss = F.cross_entropy(logits, y)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()

### 1. Инициализация

#### 1.2. Начальное распределение классов

Запустим обучение на несколько эпох и понаблюдаем за изменением ошибки:

In [ ]:
torch.manual_seed(42)
x, y = next(iter(train_loader))
input_dim = 784
hidden_dim = 128
output_dim = len(train_dataset.classes)
# создадим модель и выведем значение ошибки после инициализации
model = MLP(input_dim, hidden_dim, output_dim)
logits, h_act, h = model(x)
loss = F.cross_entropy(logits, y)
print(f"Initial loss: {loss:.4f}")

In [ ]:
n_epochs = 10
batches_per_epoch = 100
for i in range(n_epochs):
    loss = train_epoch(train_loader, model, lr=0.1, max_batches=batches_per_epoch)
    print(f"Epoch {i} loss = {loss:.4f}")

print(f"Test loss: {test_epoch(test_loader, model, max_batches=batches_per_epoch):.4f}")

Мы стартовали с очень высокого значения ошибки, но уже к третьей эпохе модель сошлась к значению около 2, после чего ошибка уже изменялась понемногу. Почему так?

Модель очень неоптимально сконфигурирована на этапе инициализации.
Начальный лосс очень далёк от ожидаемого - значит, инициализация точно плохая.

А какое ожидаемое значение ошибки?

$$CrossEntropy(\hat{y}, y) = - \sum_i y_i \log \hat{y}_i + (1 - y_i) \log(1 - \hat{y}_i)$$

Что же пошло не так?

Посмотрим, что происходит с ошибкой в зависимости от значений логитов:

А теперь посмотрим на наши логиты из модели. Что можно сказать о распределении над классами? Что это говорит о нашей инициализации?

Как нам добиться близости к нулю для логитов?

<!-- Чем занималась оптимизация первое время? Только шкалированием логитов -->

Изменилось ли значение ошибки, когда мы убрали "лёгкую часть" задачи?

#### 1.2. Значения промежуточных активаций

<img src="https://www.researchgate.net/profile/Rahul-Jayawardana/publication/350567223/figure/fig3/AS:1007855343767554@1617302847631/Fig-3-The-basic-activation-functions-of-the-neural-networksNeural-Networks.jpg" style="background:white" width="700"/>

В нашей модели мы используем нелинейность `tanh` между линейными слоями.

$$\tanh z = \frac{e^z - e^{-z}}{e^z - e^{-z}}$$

$$\frac{d(\tanh z)}{dz} = 1 - \tanh^2 z$$

Что происходит с градиентами при значениях активации близких к $0$? Близких к $1$ и $-1$?

Посмотрим на активации после нелинейности, применённой на выходы из первого слоя:

<!-- Это тот самый "vanishing gradient", о котором мы ещё услышим позже, когда будем говорить про рекуррентные сети -->

Посмотрим на масштаб проблемы:

In [ ]:
# plt.figure(figsize=(10, 20))
# plt.imshow(h_act.abs() > 0.99, cmap="gray")

Градиент будет уничтожен везде, где у нас белый пиксел.

А если найдётся целиком белый столбец?

#### 1.3. А как правильно?

Посмотрим, что происходит с распределением значений, когда мы перемножаем две матрицы, инициализированные стандартным нормальным распределением:

In [ ]:
x = torch.randn(1000, 10)
w = torch.randn(10, 200)
y =  x @ w
print(x.mean(), x.std())
print(y.mean(), y.std())
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.hist(x.flatten().tolist(), 20, density=True)
ax2.hist(y.flatten().tolist(), 20, density=True)
plt.show()


Распределение расползлось, как исправить?

**Упражнение**: какое распределение имеет $Y = X \cdot W$, где $X \in \mathbb{R}^{m \times k}$, $W \in \mathbb{R}^{k \times n}$ и $X_{ij}, W_{ij} \sim \mathcal{N}(0, 1)$?

На практике этого достаточно, но если хочется предельной точности: https://pytorch.org/docs/stable/nn.init.html
   $$\text{std} = \frac{\text{gain}(f_{act})}{\sqrt{\text{fan mode}}}$$

Пример статьи с обоснованием для ReLU и PReLU:

[Kaiming He et al. Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification](https://arxiv.org/abs/1502.01852)






Посмотрим, есть ли проблемы с встроенным слоем `torch.nn.Linear`

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
        super().__init__()
        self.l1 = nn.Linear(input_dim, hidden_dim)
        self.l2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor, Tensor]:
        h = self.l1(x.flatten(1))
        h_act = F.tanh(h)
        logits = self.l2(h_act)
        return logits, h_act, h

In [ ]:
torch.manual_seed(42)
x, y = next(iter(train_loader))
# создадим модель и выведем значение ошибки после инициализации
model = MLP(input_dim, hidden_dim, output_dim)
logits, h_act, h = model(x)
loss = F.cross_entropy(logits, y)
print(f"Initial loss: {loss:.4f}")

n_epochs = 10
batches_per_epoch = 100
for i in range(n_epochs):
    loss = train_epoch(train_loader, model, lr=0.1, max_batches=batches_per_epoch)
    print(f"Epoch {i} loss = {loss:.4f}")

print(f"Test loss: {test_epoch(test_loader, model, max_batches=batches_per_epoch):.4f}")

Явных проблем нет: инициализация по умолчанию настроена хорошо (и мы можем посмотреть, как именно)

Попробуем сделать ещё лучше?

#### Резюме
1. Проблемы с внутренними активациями возникают не только на инициализации, но и в процессе оптимизации - один большой шаг в неправильную сторону может убить нейрон (в случае с ReLU - навсегда)
2. В маленьких моделях проблемы инициализации не так критичны - сеть в итоге обучится, просто потребуется больше времени.
3. Чем глубже сеть (больше слоёв) - тем больше проблем
4. В последние годы добавилось много трюков, которые сделали инициализацию менее критичной:
   1. residual connections
   2. normalization layers (batch, layer, group)
   3. лучшие оптимизаторы (RMSProp, Adam)

### 2. Нормализация

[Ioffe, Szegedy (2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)

Хотим, чтобы активации не были ни слишком малыми, ни слишком большими, и были близки к стандартному нормальному распределению

Идея: а давайте просто возьмём активации в батче, вычтем среднее и поделим на стандартное отклонение!

<img src="https://kharshit.github.io/img/batch_normalization.png" style="background:white" width="500"/>

Какой ценой? Теперь значение активаций для одного примера уже не детерминировано - оно зависит также от других примеров в батче, который формируется случайным образом. 

Внезапно, это не так уж плохо: мы учим сеть быть устойчивой к небольшим вариациям входных данных.

Но как теперь получать предсказания для одного изолированного примера?